In [10]:
import os, sys
import numpy as np
import pandas as pd

import matplotlib
%matplotlib inline
#matplotlib.use('Agg')
import matplotlib.pyplot as plt

plt.style.use('ggplot')

caffe_root = '/home/ashu/Desktop/Thesis Work/Classifier/caffe/'
print os.getcwd()
sys.path.insert(0, caffe_root + 'python')
import caffe

import argparse

from caffe.proto import caffe_pb2
import google.protobuf as pb2
import google.protobuf.text_format

import lmdb

/home/ashu/Desktop/Thesis Work/Classifier/caffe/models/script


- Load labels and save it in list

In [37]:
obj_name = 'umbrella'
img_dir = caffe_root + 'data/gt_data/images/val/' + obj_name + '/'
label_file = caffe_root + 'data/gt_data/labels/' + obj_name + '_val.txt'

# read the labels file
with open(label_file, 'r') as f:
    data=f.readlines()

data = [x.strip() for x in data] 
print len(data)
print data[:5]

3935
['COCO_val2014_000000497032_0.jpg 1', 'COCO_val2014_000000455157_0.jpg 0', 'COCO_val2014_000000193122_0.jpg 0', 'COCO_val2014_000000193122_1.jpg 0', 'COCO_val2014_000000193122_2.jpg 0']


- Load model and prototxt

In [38]:
caffe.set_mode_gpu()

c_type = 'gt'
max_iter = '20000'
best_iter = '5000'
# Modify the paths given below
deploy_prototxt_file_path = caffe_root + 'models/' + c_type + '_classifier/' + obj_name +  \
                            '/test.prototxt' # Network definition file
# load each caffe model
caffe_model_file_path = caffe_root + 'models/gt_classifier/' + obj_name +  '/trained_model/' + \
                        max_iter + '/gt_' + obj_name + '_' +  max_iter + \
                        '_iter_' + best_iter + '.caffemodel'
net = caffe.Net(deploy_prototxt_file_path, caffe_model_file_path, caffe.TEST)

- set the transformer

In [39]:
# load the mean ImageNet image (as distributed with Caffe) for subtraction
mu = np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy')
mu = mu.mean(1).mean(1)  # average over pixels to obtain the mean (BGR) pixel values
print 'mean-subtracted values:', zip('BGR', mu)

# create transformer for the input called 'data'
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})

transformer.set_transpose('data', (2,0,1))  # move image channels to outermost dimension
transformer.set_mean('data', mu)            # subtract the dataset-mean value in each channel
transformer.set_raw_scale('data', 255)      # rescale from [0, 1] to [0, 255]
transformer.set_channel_swap('data', (2,1,0))  # swap channels from RGB to BGR

net.blobs['data'].reshape(1,        # batch size
                          3,         # 3-channel (BGR) images
                          224, 224)  # image size is 227x227

mean-subtracted values: [('B', 104.0069879317889), ('G', 116.66876761696767), ('R', 122.6789143406786)]


- iterate through images and save fc7 and its gt label

In [40]:
fc7_vectors = []
label_list = []
for val in data:
    img, lbl = val.split(' ')
    
    # load image, transform and feed forward
    image = caffe.io.load_image(img_dir + img)
    transformed_image = transformer.preprocess('data', image)
    net.blobs['data'].data[...] = transformed_image
    output = net.forward()
    
    # save generated fc7 and its state
    fc7_vectors.append(net.blobs['fc7'].data.copy())
    label_list.append(lbl)
    
print 'fc7: \n', fc7_vectors[-4:]
#print 'labels ', label_list[:30]

fc7: 
[array([[ 0.       ,  0.       ,  0.       , ...,  0.1332878,  0.       ,
         1.9835149]], dtype=float32), array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  1.83259046]], dtype=float32), array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  1.22264707]], dtype=float32), array([[ 3.62063861,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.20422645]], dtype=float32)]


In [41]:
fc7_dir = caffe_root + 'models/' + c_type + '_classifier/' + obj_name + '/fc7/'
if not os.path.exists(fc7_dir):
    os.makedirs(fc7_dir)
    
fc7_file = fc7_dir + c_type + '_fc7_' + obj_name + '.pkl'
lbl_file = fc7_dir + c_type + '_label_' + obj_name + '.pkl'

print fc7_dir, fc7_file, lbl_file

/home/ashu/Desktop/Thesis Work/Classifier/caffe/models/gt_classifier/umbrella/fc7/ /home/ashu/Desktop/Thesis Work/Classifier/caffe/models/gt_classifier/umbrella/fc7/gt_fc7_umbrella.pkl /home/ashu/Desktop/Thesis Work/Classifier/caffe/models/gt_classifier/umbrella/fc7/gt_label_umbrella.pkl


In [ ]:
import pickle

# dump fc7 features
fileObject = open(fc7_file,'wb')
pickle.dump(fc7_vectors,fileObject)
fileObject.close()

# dump state labels
fileObject = open(lbl_file,'wb')
pickle.dump(label_list,fileObject)
fileObject.close()